
    
- 前置作業: 
    - https://ithelp.ithome.com.tw/articles/10229943
    - https://developers.line.biz/en/docs/messaging-api/overview/#next-steps
  
- line-gituhb
    - linebot sdk: https://github.com/line/line-bot-sdk-python
    - flask-kitchensink: https://github.com/line/line-bot-sdk-python/blob/5ab6ba225495248c52eb3aa728da427bedffb37d/examples/flask-kitchensink/app.py
    - message content: https://github.com/line/line-bot-sdk-python/blob/5ab6ba225495248c52eb3aa728da427bedffb37d/linebot/models/responses.py
    - reply_message: https://github.com/line/line-bot-sdk-python/blob/5ab6ba225495248c52eb3aa728da427bedffb37d/linebot/api.py

- python decorator: https://foofish.net/python-decorator.html

- 拿使用者傳來的照片: https://ithelp.ithome.com.tw/articles/10244895

- 開發LINE聊天機器人不可不知的十件事: https://engineering.linecorp.com/zh-hant/blog/line-device-10/

# deploy.sh

In [ ]:
gcloud app deploy app.yaml \
--project="praxis-electron-301404" \
--verbosity="info"

# app.yaml

In [ ]:
service: line-bot

instance_class: F1
    
runtime: python38

entrypoint: gunicorn --bind :$PORT main:app

automatic_scaling:
    min_instances: 0
    max_instances: 5

# requirements.txt

In [ ]:
pip==20.2.4
Flask==1.1.2
gunicorn==19.9.0
line-bot-sdk==1.18.0
google-cloud-datastore==2.0.1
pandas==1.2.0

In [ ]:
!pip show pandas

# main.py

In [ ]:
from flask import Flask, request, abort, jsonify
from utils import manager

from linebot.exceptions import (
    LineBotApiError, InvalidSignatureError
)

from linebot.models import (
    MessageEvent, TextMessage, StickerMessage, LocationMessage, ImageMessage, VideoMessage, AudioMessage, FileMessage
)

app = Flask(__name__)

#####################################################################
managerUtil = manager.ManagerUtil()
managerUtil.setup_line_bot()
handler = managerUtil.handler
#####################################################################

@app.route('/')
def index():
    return "hihi"

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except LineBotApiError as e:
        print("Got exception from LINE Messaging API: %s\n" % e.message)
        for m in e.error.details:
            print("  %s: %s" % (m.property, m.message))
        print("\n")
    except InvalidSignatureError:
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=(TextMessage, StickerMessage, LocationMessage, ImageMessage, VideoMessage, AudioMessage, FileMessage))
def handle_message(event):
    managerUtil.setup_event(event)
    managerUtil.handle_message()
    
if __name__ == "__main__":
    app.run(host='127.0.0.1', port=8080, debug=True)    
#     app.run(host="0.0.0.0", port=8787, debug=True)


# utils

## member.py

In [ ]:
kind = LineUser
name = userID

userID = line_user_id
createTime = now

# 開通之後要更新
ticket_level = A
search_quota = 20
ticketID = None
ticket_start_time = now
ticket_end_time = now + 30days



In [6]:
import pytz
import pandas as pd
from datetime import datetime, timedelta, timezone
from google.cloud import datastore

class MemberUtil:
    def __init__(self, default_search_quota=50, default_days=60):
        self.default_search_quota = default_search_quota
        self.default_days = default_days
        self.datastore_client = datastore.Client()
        self.setup_rate_limit_table()
        
    def setup_user(self, studentID):
        name = studentID
        kind = 'LineUser'
        key = self.datastore_client.key(kind, name)
        user_entity = self.datastore_client.get(key)
        if user_entity == None:
            user_entity = self.create_user(studentID)
        self.user_entity = user_entity
                
    def create_user(self, studentID):
        hours = self.default_days*24
        current_time = self.get_current_time()
        rate_limit_table = self.rate_limit_table
        
        name = studentID
        kind = 'LineUser'
        key = self.datastore_client.key(kind, name)
        entity = datastore.Entity(key=key)
        
        entity['userID'] = studentID
        entity['createTime'] = current_time
        
        entity['ticketID'] = None
        entity['ticket_level'] = None
        entity['ticket_search_quota'] = self.default_search_quota
        entity['ticket_start_time'] = current_time
        entity['ticket_end_time'] = current_time + timedelta(hours=hours)
        
        entity['is_normal'] = 1
        for i in range(len(rate_limit_table)):
            end_time_name = rate_limit_table.loc[i, 'name'] + '_end_time'
            minute = int(rate_limit_table.loc[i, 'minute'])
            end_time = current_time + timedelta(minutes=minute) 
            
            search_quota_name = rate_limit_table.loc[i, 'name'] + '_search_quota'
            search_quota = int(rate_limit_table.loc[i, 'quota'])
            
            entity[end_time_name] = end_time
            entity[search_quota_name] = search_quota

        self.datastore_client.put(entity)
        return entity
    
    def get_current_time(self):
        current_time = datetime.now().replace(tzinfo=timezone.utc)
        return current_time
    
    def get_user_type(self):
        current_time = self.get_current_time()
        
        ticket_end_time = self.user_entity['ticket_end_time']
        ticket_search_quota = self.user_entity['ticket_search_quota']
        is_normal = self.user_entity['is_normal']
        
        if is_normal:
            if ticket_search_quota == 0 or current_time > ticket_end_time:
                user_type = 'pay'
            else:
                user_type = 'normal'
        else:
            user_type = 'abnormal'
        return user_type
    
    def update_GDS_LineUser(self):
        self.update_GDS_LineUser_search_quota()
        self.update_GDS_LineUser_rate_limit_end_time()
        self.datastore_client.put(self.user_entity)
    
    def update_GDS_LineUser_search_quota(self):
        rate_limit_table = self.rate_limit_table
        self.user_entity['ticket_search_quota'] -= 1
        for i in range(len(rate_limit_table)):
            search_quota_name = rate_limit_table.loc[i, 'name'] + '_search_quota'
            self.user_entity[search_quota_name] -= 1
            
            # prevent many request
            if self.user_entity[search_quota_name] == 0:
                self.user_entity['is_normal'] = 0
                
            
    def update_GDS_LineUser_rate_limit_end_time(self):
        rate_limit_table = self.rate_limit_table
        current_time = self.get_current_time()        
        for i in range(len(rate_limit_table)):
            name = rate_limit_table.loc[i, 'name']
            minute = int(rate_limit_table.loc[i, 'minute'])
            search_quota = int(rate_limit_table.loc[i, 'quota'])
            
            end_time_name = name + '_end_time'
            search_quota_name = name + '_search_quota'
            
            end_time = self.user_entity[end_time_name]
            if current_time > end_time:
                end_time = current_time + timedelta(minutes=minute) 
                self.user_entity[end_time_name] = end_time
                self.user_entity[search_quota_name] = search_quota
                        
    def setup_rate_limit_table(self):
        rate_limit_table = pd.DataFrame()
        rate_limit_table['name'] = ['minute1', 'minute5']
        rate_limit_table['minute'] = [1, 5]
        rate_limit_table['quota'] = [5, 10]
        self.rate_limit_table = rate_limit_table
        

In [7]:
memberUtil = MemberUtil()


In [11]:
studentID = 'U9ee09c740c77152094d16a2c2b5260a6'
# memberUtil.create_user(studentID)
memberUtil.setup_user(studentID)


In [12]:
memberUtil.update_GDS_LineUser()

## ticket.py

In [ ]:
from google.cloud import datastore
import pandas as pd
from datetime import datetime, timedelta, timezone
import pytz

class TicketUtil:
    def __init__(self):
        self.datastore_client = datastore.Client()
        
    def get_ticket(self, ticketID):
        key = self.datastore_client.key('Ticket', ticketID)
        ticket = self.datastore_client.get(key)
        return ticket
    
    def ticket_is_used(self, ticket):
        userID = ticket['userID']
        is_used = (userID!=None)
        return is_used
    
    def get_current_time(self):
        current_time = datetime.now().replace(tzinfo=timezone.utc)
        return current_time
    
    def get_activate_info(self, studentID, ticket):
        ticket_level = ticket['ticket_level']
        
        ticket_level_table = self.get_ticket_level_table()
        c = (ticket_level_table.ticket_level ==ticket_level)
        ticket_level_table = ticket_level_table[c]
        
        days = ticket_level_table.days.values[0]
        hours = int(days * 24)
        ticket_search_quota = int(ticket_level_table.ticket_search_quota.values[0]) #! 要累加上一期的嗎
        
        ticketID = ticket['ticketID']
        ticket_start_time = self.get_current_time()
        ticket_end_time = ticket_start_time + timedelta(hours=hours)
        
        activate_info = {}
        activate_info['userID'] = studentID
        activate_info['ticket_level'] = ticket_level
        activate_info['ticket_search_quota'] = ticket_search_quota
        activate_info['ticketID'] = ticketID
        activate_info['ticket_start_time'] = ticket_start_time
        activate_info['ticket_end_time'] = ticket_end_time
        return activate_info

    def update_GDS_Ticket(self, activate_info):
        ticketID = activate_info['ticketID']
        key = self.datastore_client.key('Ticket', ticketID)
        entity = self.datastore_client.get(key)
        entity['userID'] = activate_info['userID']
        entity['ticket_start_time'] = activate_info['ticket_start_time']
        entity['ticket_end_time'] = activate_info['ticket_end_time']
        self.datastore_client.put(entity)
        
    def update_GDS_LineUser(self, activate_info):
        userID = activate_info['userID']
        key = self.datastore_client.key('LineUser', userID)
        entity = self.datastore_client.get(key)
        entity['ticket_level'] = activate_info['ticket_level']
        entity['ticket_search_quota'] = activate_info['ticket_search_quota']
        entity['ticketID'] = activate_info['ticketID']
        entity['ticket_start_time'] = activate_info['ticket_start_time']
        entity['ticket_end_time'] = activate_info['ticket_end_time']
        self.datastore_client.put(entity)
    
    def get_ticket_level_table(self):
        df = pd.DataFrame()
        df['ticket_level'] = ['A', 'B']
        df['days'] = [30, 30]
        df['ticket_search_quota'] = [20, 150] #[20, 150]
        return df
        
        

In [ ]:
ticketUtil = TicketUtil()


In [ ]:
ticketUtil.get_ticket_level_table()

In [ ]:
ticketID = '1d6a52a1a688406ba2376625ded49301'
ticket = ticketUtil.get_ticket(ticketID)
ticket

In [ ]:
ticketUtil.ticket_is_used(ticket)

In [ ]:
studentID = 'U9ee09c740c77152094d16a2c2b5260a6'

In [ ]:
activate_info = ticketUtil.get_activate_info(studentID, ticket)
activate_info

In [ ]:
ticketUtil.update_GDS_Ticket(activate_info)

In [ ]:
ticketUtil.update_GDS_LineUser(activate_info)

## search.py

In [ ]:
import requests
import base64
import json

class SearchUtil:
    def __init__(self):
        pass
    
    def search_question(self, studentID, b2_bytes):
        # imageData
        
        b64_bytes = self.b2bytes_to_b64bytes(b2_bytes)
        b64_text = b64_bytes.decode("utf-8") # bytes2text(b64_bytes)

        request_body = {}
        request_body['studentID'] = studentID # 'student-linebot'
        request_body['imageData'] = b64_text 

        url = "https://search-question-dot-praxis-electron-301404.dt.r.appspot.com/search_question"
        payload = json.dumps(request_body)
        headers = {'content-type': "application/json"}
        response = requests.request("POST", url, data=payload, headers=headers)
        return_body = json.loads(response.text)

        answerArray = return_body['answerArray']
        answer = answerArray[0]

        q_url = self.get_questionImagePath(answer)
        a_url = self.get_answerImagePath(answer)

        return q_url, a_url

    def b2bytes_to_b64bytes(self, b2bytes):
        b64bytes = base64.b64encode(b2bytes)
        return b64bytes

    def get_questionImagePath(self, answer):
        questionID = answer['questionID']
        url = 'https://storage.googleapis.com/easy668/imageFile/%s'%(questionID)
        return url

    def get_answerImagePath(self, answer):
        a1 = {}
        a1['answerID'] = answer['answerID']
        a1['stepIDArray'] = answer['stepIDArray'] 
        request_body = [a1]

        url = 'https://asia-northeast2-praxis-electron-301404.cloudfunctions.net/get_stepArray'
        payload = json.dumps(request_body)
        headers = {'content-type': "application/json"}
        response = requests.request("POST", url, data=payload, headers=headers)
        return_body = json.loads(response.text)

        answerArray = return_body['result']
        answer = answerArray[0]
        stepArray = answer['stepArray']
        step = stepArray[0]
        stepDetailArray = step['stepDetailArray']
        stepDetail = stepDetailArray[0]
        answerImagePath = stepDetail['answserImagePath'] # answserImagePath 字打錯
        return answerImagePath
    
        

In [ ]:
searchUtil = SearchUtil()

In [ ]:
path = './a.png'
with open(path, 'rb') as f:
    b2_bytes = f.read()
    
len(b2_bytes)

In [ ]:
studentID = 'U9ee09c740c77152094d16a2c2b5260a6'

In [ ]:
searchUtil.search_question(studentID, b2_bytes)

## manager.py

In [ ]:
from utils import member, ticket, search
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.models import (
    MessageEvent, TextMessage, StickerMessage, LocationMessage, ImageMessage, VideoMessage, AudioMessage, FileMessage, TextSendMessage, ImageSendMessage
)


class ManagerUtil:
    def __init__(self):
        self.memberUtil = member.MemberUtil()
        self.ticketUtil = ticket.TicketUtil()
        self.searchUtil = search.SearchUtil()
        self.line_teacher = 'https://lin.ee/vBlZxBF'
        self.line_service = 'https://lin.ee/XpbrUnI'
        
    def setup_line_bot(self):
        channel_secret = '7f8ac1b4e86b97aff37c09c5ee2ebead'
        self.handler = WebhookHandler(channel_secret)

        channel_access_token = 'YJhR/xR7YkbnhVaxKzaPehNFh5w/RR+Ye7cUqNxfxQYAXDb4R4ZgKrDu1bgElC+tSxFzPWNdDmcC8gqEC607TTHNW37Y7ApHWxIxEv60+mEoXrNIckZvRtyTggEKml+ZzJm+GtOZJPyU0bJr6gEuAAdB04t89/1O/w1cDnyilFU='
        self.line_bot_api = LineBotApi(channel_access_token)
        
    def setup_event(self, event):
        self.event = event
        self.reply_token = event.reply_token
        self.studentID = event.source.user_id
        self.memberUtil.setup_user(self.studentID) # O(1)
        
    def handle_message(self):
        user_type = self.memberUtil.get_user_type() # pay, normal, abnormal
        if user_type == 'normal':
            if isinstance(self.event.message, ImageMessage):
                self.handle_image()
            elif isinstance(self.event.message, TextMessage):  
                self.handle_text(text="可以把不會的題目拍起來傳給我們，我們就會回傳解答給你喔～")
            else:
                self.reply_text(text="可以把不會的題目拍起來傳給我們，我們就會回傳解答給你喔～") 
                
        elif user_type == 'pay':
            self.handle_text(text="填問卷拿好學問序號，就能繼續找解答囉～ https://forms.gle/vNuyhQCTCrs14hkN9")
            
        elif user_type == 'abnormal':
            self.reply_text(text="系統異常，請洽詢好學問客服～ %s"%(self.line_service)) 
    
    def handle_image(self):
        message_content = self.line_bot_api.get_message_content(self.event.message.id)
        b2_bytes = message_content.content 

        # 確認server有開
        try:
            q_url, a_url = self.searchUtil.search_question(self.studentID, b2_bytes)

            m1 = ImageSendMessage(q_url, q_url)
            m2 = ImageSendMessage(a_url, a_url)
            m3 = TextSendMessage(text="如果看不懂的話，可以到這裡問老師～ %s"%(self.line_teacher))

            messages = [m1, m2, m3]
            self.line_bot_api.reply_message(self.reply_token, messages)

            # 回傳之後再update
            self.memberUtil.update_GDS_LineUser() 

        except Exception as e:
            self.reply_text(text="搜尋引擎休息中，可以到這裡問老師～ %s"%(self.line_teacher)) 
            
    def handle_text(self, text=""):
        # 確認是不是ticket，不是的話就回同一句話
        if isinstance(self.event.message, TextMessage):  
            ticketID = self.event.message.text
            ticket = self.ticketUtil.get_ticket(ticketID)
            if ticket!=None:
                self.handle_ticket(ticket)
            else:
                self.reply_text(text=text) 
        else:
            self.reply_text(text=text) 
            
    def handle_ticket(self, ticket):
        is_used = self.ticketUtil.ticket_is_used(ticket)
        if is_used == True:
            self.reply_text(text="這個好學問序號已經使用過囉～") 
        else:
            activate_info = self.ticketUtil.get_activate_info(self.studentID, ticket)

            ticketID = activate_info['ticketID']
            ticket_start_time = activate_info['ticket_start_time'].strftime('%Y-%m-%d')
            ticket_end_time = activate_info['ticket_end_time'].strftime('%Y-%m-%d')

            text = '好學問序號 %s 已經開通，使用期間為 %s 至 %s。'%(ticketID, ticket_start_time, ticket_end_time)
            self.reply_text(text=text) 

            # 回傳之後再update
            self.ticketUtil.update_GDS_Ticket(activate_info)
            self.ticketUtil.update_GDS_LineUser(activate_info)
            
    def reply_text(self, text=""):
        text_message = TextSendMessage(text=text)
        messages = [text_message]
        self.line_bot_api.reply_message(self.reply_token, messages)
        
        
        

# api

http://35.234.31.76:8787/fuck_test

https://line-bot-dot-praxis-electron-301404.dt.r.appspot.com/fuck_test

https://line-bot-dot-praxis-electron-301404.dt.r.appspot.com/callback

# request

In [ ]:
{
    "b64_text": b64_text
}

In [ ]:
POST /callback HTTP/1.1
X-Line-Signature: j9p1sXsb0yCyIEE8cEsmHbp9eHc85P2DQBVH1RKiQLk=
Content-Type: application/json;charset=UTF-8
Content-Length: 223
Host: callback.sample.com
Accept: */*
User-Agent: LineBotWebhook/1.0

{
    "events":[
        {
            "type":"message",
            "replyToken":"1234567890abcdef1234567890abcdef",
            "timestamp":1500052665000,
            "source":{
                "userId":"U1234567890abcdef1234567890abcdef",
                "type":"user"
            },
            "message":{
                "type":"audio",
                "id":"1234567890123"
            }
        }
    ]
}

# b2_bytes

In [ ]:
path = './a.png'
with open(path, 'rb') as f:
    b2bytes = f.read()
type(b2bytes), len(b2bytes)

In [ ]:
import base64

def b2bytes_to_b64bytes(b2bytes):
    b64bytes = base64.b64encode(b2bytes)
    return b64bytes

In [ ]:
b64bytes = b2bytes_to_b64bytes(b2bytes)
type(b64bytes), len(b64bytes)

In [ ]:
b64text = b64bytes.decode("utf-8")
type(b64text), len(b64text)

In [ ]:
# b64text

# get line id

- 沒提供lineid: https://github.com/line/line-bot-sdk-java/issues/266

In [ ]:
from linebot import (
    LineBotApi, WebhookHandler
)

In [ ]:
channel_access_token = 'YJhR/xR7YkbnhVaxKzaPehNFh5w/RR+Ye7cUqNxfxQYAXDb4R4ZgKrDu1bgElC+tSxFzPWNdDmcC8gqEC607TTHNW37Y7ApHWxIxEv60+mEoXrNIckZvRtyTggEKml+ZzJm+GtOZJPyU0bJr6gEuAAdB04t89/1O/w1cDnyilFU='
channel_secret = '7f8ac1b4e86b97aff37c09c5ee2ebead'

line_bot_api = LineBotApi(channel_access_token)
handler = WebhookHandler(channel_secret)

user_id = 'U9ee09c740c77152094d16a2c2b5260a6'
profile = line_bot_api.get_profile(user_id)
profile


# error

In [ ]:
從bot mode改成chat mode之後 要把Use webhook打開

In [ ]:
如果是要給一個特定的台灣時間，要另外設定tzinfo，其他例如datetime.now這種就不用另外設定